In [3]:
import mujoco as mu
import os
import numpy as np
import mujoco.viewer as vi
import mediapy as media

In [4]:
xml4="""
<mujoco>
	<option timestep="0.0001" integrator="RK4" >
		<flag sensornoise="enable" energy="enable" contact="enable" />
	</option>
	<worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
		<geom type="plane" size="1 1 0.1" rgba=".9 0 0 1"/>
		<body pos="0 0 1.25" euler="0 180 0">
			<joint name="pin" type="hinge" axis = "0 -1 0" pos="0 0 -0.5"/>
			<geom type="cylinder" size="0.05 0.5" rgba="0 .9 0 1" mass="1"/>
			<body pos="0 0.1 1" euler="0 0 0">
				<joint name="pin2" type="hinge" axis = "0 -1 0" pos="0 0 -0.5"/>
				<geom type="cylinder" size="0.05 0.5" rgba="0 0 .9 1" mass="1"/>
			</body>
		</body>
	</worldbody>

	<actuator>
		<motor joint="pin" name="torque" gear="1" ctrllimited="true" ctrlrange="-100 100" />
		<motor joint="pin2" name="torque2" gear="1" ctrllimited="true" ctrlrange="-100 100" />
	</actuator>
    
</mujoco>
"""

xml44 = """
<mujoco model="base">
  <actuator>
      <motor joint='t_joint_x' ctrllimited='true' ctrlrange='-2.0 2.0'/>
      <motor joint='t_joint_y' ctrllimited='true' ctrlrange='-2.0 2.0'/>
  </actuator>

  <equality>
    <weld active="true" body1="cursor0" body2="block1" solimp="1 1 0.5" solref="0.01 0.3" />
  </equality>

  <worldbody>
    <geom condim="3" friction="2.0 0.005 0.0001" name="FLOOR" pos="0 0 0" rgba="0.7 0.7 0.7 1" size="0.5 0.5 0.0625" type="plane" />
    <light castshadow="true" diffuse=".8 .8 .8" dir=".5 -.3 -.8" directional="true" pos="0 0 2.0" specular="0.3 0.3 0.3" />

    <body name="cursor0" pos="0.2 -0.3 0.05">
      <geom condim="3" gap="10" margin="0.05" name="cursor0" rgba="1 0 0 0.5" size="0.05 0.05 0.05" type="box" />
      <joint type='slide' axis='1 0 0' damping='0.5' pos='0 0 0' name='t_joint_x' range='-10.0 10.0'/>
      <joint type='slide' axis='0 1 0' damping='0.5' pos='0 0 0' name='t_joint_y' range='-10.0 10.0'/>
    </body>

    <body name="block1" pos="0.201 0.0 0.05" quat="1 0 0 0">
      <geom density="100" friction="1 10 0.5"  name="block1-0" rgba="0.82 0.71 0.55 1" size="0.1 0.02 0.05" solref="0.001 1" type="box" />
      <joint damping="0.0001" name="block1" type="free" />
    </body>

  </worldbody>
</mujoco>
"""

In [5]:
xml2 = """
<mujoco>
  <compiler autolimits="true"/>
  <!--
  Adding a high fluid viscosity and using implicit integration for extra stabillity.
  Extra stabillity is required because the abstract "arm" model is not very realistic.
  (e.g. 4 consecutive ball joints is not a realistic kinematic design)
  -->
  <option viscosity="10" integrator="implicit">
    <flag gravity="disable" contact="disable"/>
  </option>

  <statistic meansize=".05"/>

  <default>
    <default class="translation">
      <position kp="100" kv="10" ctrlrange="-.25 .25"/>
    </default>
    <default class="rotation">
      <!--
      Note that the rotational control range is purposefully limited to (-pi/2, pi/2) to avoid the
      documented instabillity near pi, which is due to taking quaternion differences.
      Increase this range to pi or bigger in order to see the instabillity.
      See here for more details https://mujoco.readthedocs.io/en/latest/XMLreference.html#actuator
      -->
      <position kp=".5" kv=".05" ctrlrange="-1.571 1.571"/>
    </default>
    <!--
    Clamping the total actuator torque at the joints means that the motion produced by the Cartesian
    commands is achievable by individual joint actuators with the specified torque limits.
    See https://mujoco.readthedocs.io/en/latest//modeling.html#actuator-force-clamping
    -->
    <joint stiffness="1e-1" actuatorfrcrange="-1 1"/>
    <site type="box" size=".012 .012 .012" rgba=".7 .7 .8 1"/>
  </default>

  <worldbody>
    <light pos="0 0 2"/>
    <geom type="box" size=".25 .25 .01" pos="0 0 -.01"/>
    <site name="reference" pos="0 0 .25"/>
    <body name="arm" pos="-.25 .25 0">
      <joint axis="1 0 0"/>
      <joint axis="0 1 0"/>
      <geom type="box" size=".01" fromto="0 0 0 0 0 .25"/>
      <body pos="0 0 .25">
        <joint axis="0 1 0"/>
        <joint axis="0 0 1"/>
        <geom type="box" size=".01" fromto="0 0 0 .25 0 0"/>
        <body pos=".25 0 0">
          <joint axis="1 0 0"/>
          <joint axis="0 0 1"/>
          <geom type="box" size=".01" fromto="0 0 0 0 -.2 0"/>
          <body pos="0 -.2 0">
            <joint axis="1 0 0"/>
            <joint axis="0 0 1"/>
            <joint axis="0 1 0"/>
            <geom type="box" size=".01" fromto="0 0 0 0 -.05 0"/>
            <site name="end_effector" pos="0 -.05 0"/>
          </body>
        </body>
      </body>
    </body>
  </worldbody>

  <actuator>
    <position name="x"  site="end_effector" refsite="reference" gear="1 0 0 0 0 0" class="translation"/>
    <position name="y"  site="end_effector" refsite="reference" gear="0 1 0 0 0 0" class="translation"/>
    <position name="z"  site="end_effector" refsite="reference" gear="0 0 1 0 0 0" class="translation"/>
    <position name="rx" site="end_effector" refsite="reference" gear="0 0 0 1 0 0" class="rotation"/>
    <position name="ry" site="end_effector" refsite="reference" gear="0 0 0 0 1 0" class="rotation"/>
    <position name="rz" site="end_effector" refsite="reference" gear="0 0 0 0 0 1" class="rotation"/>
  </actuator>
</mujoco>
"""

In [6]:
xml_box = """
<mujoco>
  <default>
    <default class="translation">
      <position kp="100" kv="10" ctrlrange="-.5 .5"/>
    </default>
  </default>
</mujoco>
"""

In [7]:
xml6 = """
<mujoco>
    <compiler autolimits="true"/>
    <asset>
        <!-- Define ground material -->
        <material name="ground" rgba="0.8 0.9 0.8 1"/>
    </asset>

    <option gravity ="0 0 0"/>
    
  <default>
    <default class="translation">
      <position kp="100" kv="10" ctrlrange="-.5 .5"/>
    </default>
    <default class="rotation">
      <position kp=".5" kv=".05" ctrlrange="-1.571 1.571"/>
    </default>
  </default>

  <option viscosity="10" integrator="implicit">
    <flag contact="disable"/>
  </option>

    <visual>
      <headlight ambient="0.5 0.5 0.5"/>
    </visual>

    <worldbody>
        <!-- Define the ground -->
        <geom name="ground" type="plane" size="1 1 0.1" material="ground"/>

        <!-- Define the box -->
        <body name="box" pos="0 0 0.5">
            <site name="anchor1" pos="0 0 0.5"/>
            <!--joint name="pin" type="hinge" axis = "0 -1 0" pos="0 0 -0.5"/-->
            <geom name="box_geom" type="box" size="0.1 0.1 0.1" rgba="0.5 0.5 0.5 1"/>

            <body name="rod1" pos="0 -0.1 0.5" euler="0 0 0">
              <joint name="pin2" type="hinge" axis = "0 -1 0" pos="0 0 -0.5"/>
				      <geom type="cylinder" size="0.05 0.5" rgba="0 0 .9 1" mass="1"/>
              <joint name="pin2s" type="slide" axis = "0 0 1" pos="0 0 -0.5"/>
              <site name="site2" pos="0 -0.1 0.5"/>

              <body name="rod12" pos="0 -0.1 0.5" euler="0 0 0">
                <joint name="pin2_1" type="slide" axis = "0 0 1" pos="0 0 -0.5"/>
				        <geom type="cylinder" size="0.05 0.5" rgba="0 0 .9 1" mass="1"/>
                <site name="site1" pos="0 0 1"/>
              </body>
            </body>
        </body>
    </worldbody>
    
    <equality>
      <connect name="connect" body1="box" body2="rod1" anchor="0 0 0.5"/>
        <!--weld name="weld1" body1="rod12" anchor="0 0 1.5"/-->
        <!--weld name="weld2" body1="rod1" body2="rod12"/-->
    </equality> 

  <actuator>
    <position name="move1" site="site1" refsite="anchor1" gear="0 0 1 0 0 0" class="translation"/>
    <motor name="move3" site="site2" refsite="anchor1" gear="0 0 1 0 0 0" class="translation"/>
    <!--position name="move2" joint="" slidersite="site1"/-->
  </actuator>
    
  <!--
	<actuator>
		<motor joint="pin" name="torque" gear="1" ctrllimited="true" ctrlrange="-100 100" />
		<motor joint="pin2" name="torque2" gear="1" ctrllimited="true" ctrlrange="-100 100" />
	</actuator>
  -->
  </mujoco>
"""

In [8]:
#1 spinny circle
xmlv2="""
<mujoco>
  <compiler autolimits="true"/>

  <option integrator="implicitfast"/>

  <worldbody>
    <geom type="plane" size="1 1 .01"/>
    <light pos="0 0 2"/>
    <body pos="0 0 .3">
      <joint name="hinge" damping=".01" actuatorfrcrange="-.4 .4"/>
      <geom type="capsule" size=".01" fromto="0 0 0 .2 0 0"/>
      <geom size=".03" pos=".2 0 0"/>
    </body>
  </worldbody>

  <actuator>
    <motor name="motor" joint="hinge" ctrlrange="-1 1"/>
    <damper name="damper" joint="hinge" kv="10" ctrlrange="0 1"/>
  </actuator>

  <sensor>
    <actuatorfrc name="motor" actuator="motor"/>
    <actuatorfrc name="damper" actuator="damper"/>
    <jointactuatorfrc name="hinge" joint="hinge"/>
  </sensor>
</mujoco>"""

#Spinny objects based on sites
xmlv3="""
<mujoco>
  <option density="1000">
    <flag gravity="disable"/>
  </option>

  <worldbody>
    <geom type="plane" size="10 10 .1" rgba="1 1 1 1"/>
    <light pos="0 0 20"/>
    <body name="body_stateless" pos="0 0 .5">
      <freejoint/>
      <geom type="box" size=".5 .5 .1" pos="0 0 .4"/>
      <site name="capsule" pos=".5 .5 .5" rgba="0 0 1 1" size=".1 .1" type="capsule"/>
      <site name="ellipsoid" pos=".5 -.5 .5" rgba="0 0 1 1" size=".1 .3 .1" type="ellipsoid" euler="0 0 -45"/>
      <site name="cylinder" pos="0 0 .5" rgba="0 0 1 1" size=".2 .1" type="cylinder"/>
      <site name="box" pos="-.5 -.5 .5" rgba="0 0 1 1" size=".1 .1 .2" type="box"/>
    </body>
  </worldbody>

  <actuator>
    <general name="ellipsoid" site="ellipsoid" gear="0 1 0 0 0 0" ctrllimited="true" ctrlrange="-100 100"/>
    <general name="cylinder" site="cylinder" gear="0 0 0 0 0 1" ctrllimited="true" ctrlrange="-100 100"/>
    <general name="box" site="box" dyntype="integrator" gear="0 0 1 0 0 0" ctrllimited="true" ctrlrange="-100 100"/>
    <general name="capsule" site="capsule" dyntype="integrator" gear="0 0 1 0 0 0" ctrllimited="true" ctrlrange="-100 100"/>
  </actuator>
</mujoco>"""


#
xmlv4="""
<mujoco>

  <compiler angle="radian"/>

  <option timestep="0.0001" integrator="RK4">
    <flag gravity="disable" contact="disable"/>
  </option>

  <size memory="1M"/>

  <statistic meansize="0.193896" extent="1" center="0 0 0.3"/>

  <default>
    <joint actuatorfrcrange="-18 18"/>
    <default class="cartesian">
      <general biastype="affine" dynprm="0.189824" gainprm="51.073" biasprm="0 -51.073 -1.03446"/>
    </default>
  </default>

  <asset>
    <texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="0.2 0.3 0.4" rgb2="0.1 0.2 0.3" markrgb="0.8 0.8 0.8" width="200" height="200"/>
    <material name="groundplane"  texture="groundplane" texuniform="true" texrepeat="2 2" reflectance="0.2"/>
  </asset>

  <worldbody>
    <geom size="0 0 0.1" type="plane" material="groundplane"/>
    <light pos="0 0 1.5" dir="0 0 -1"/>
    <body pos="0 0 0.6">
      <joint type="free"/>

      <body name="base">
        <geom size="0.0606401 0.116554 0.163268" pos="-0.00170944 -0.000675197 0.00191671" quat="-0.000920387 0.70756 -0.000583555 0.706653" type="box"/>
        <site name="base_frame"/>
        <body pos="0.171671 -0.0907507 -9.8e-06" quat="0.5 -0.5 0.5 -0.5">
          <joint axis="0 0 1" range="-1.0472 1.0472" armature="0.0109013" damping="0.0239" frictionloss="0.1334"/>
          <geom size="0.0324681 0.0432267 0.0500827" pos="-0.00997157 -1.75508e-05 0.0591165" quat="0.99973 -0.0108474 -0.0205183 -0.000825809" type="box"/>
          <body pos="0.03085 0 0.065" quat="0 -0.707107 0 -0.707107">
            <joint axis="0 0 1" range="-1.54706 3.02902" armature="0.0109013" damping="0.0239" frictionloss="0.1334"/>
            <geom size="0.0230114 0.044369 0.0456418" pos="0.00023322 7.40609e-05 0.0232897" quat="0.646671 -0.288993 -0.645777 -0.285081" type="box"/>
            <geom size="0.0165189 0.0320859 0.136968" pos="-0.0789385 0.0134066 0.0697267" quat="0.0465854 -0.70695 -0.00148786 0.705726" type="box"/>
            <body pos="-0.19 0 0.069575" quat="0 0 -1 0">
              <joint axis="0 0 1" range="0 2.44346" armature="0.0109013" damping="0.0239" frictionloss="0.1334"/>
              <geom size="0.0119255 0.0121628 0.0197376" pos="-0.211513 -0.0748022 1.73156e-06" quat="0.642033 0.296302 -0.642156 0.296031" type="box"/>
              <geom size="0.008597 0.0161382 0.11823" pos="-0.0783394 -0.0253533 -1.08631e-05" quat="0.702549 0.0808911 -0.702432 0.0804412" type="box"/>
              <site name="foot_front_right" pos="-0.21425 -0.0779806 0" quat="0.664463 0.664463 -0.241845 -0.241845"/>
            </body>
          </body>
        </body>
      </body>
    </body>
  </worldbody>

  <actuator>
    <general name="x linear" class="cartesian" site="foot_front_right" refsite="base_frame" ctrlrange="-5 5" gear="1 0 0 0 0 0"/>
    <general name="y rotational" class="cartesian" site="foot_front_right" refsite="base_frame" ctrlrange="-5 5" gear="0 0 0 0 .1 0"/>
  </actuator>

  <sensor>
    <subtreeangmom body="base"/>
    <subtreelinvel body="base"/>
  </sensor>
</mujoco>"""

xmlv5 = """
<mujoco>
  <compiler autolimits="true"/>

  <option density="1000" timestep="0.05" integrator="implicitfast" />

  <default>
    <geom type="sphere" size="0.1" rgba="1 1 1 1"/>
    <joint type="slide" axis="0 0 1"/>
    <general ctrlrange="-1 1"/>
    <default class="filter">
      <general dyntype="filter" gear="100"/>
    </default>
    <default class="filterexact">
      <general dyntype="filterexact" gear="100"/>
    </default>
    <default class="integrator">
      <general dyntype="integrator" actrange="0 2" gear="100"/>
      <default class="integrator_actearly">
        <general actearly="true"/>
      </default>
    </default>
  </default>

  <worldbody>
    <geom type="plane" size="10 10 .1" rgba="1 1 1 1"/>
    <light pos="0 0 20"/>

    <body pos="-0.3  0 0.1"><joint name="filter_actearly"/><geom/></body>
    <body pos="-0.1 0 0.1"><joint name="filter"/><geom/></body>

    <body pos="0.1  0 0.1"><joint name="filterexact_actearly"/><geom/></body>
    <body pos="0.3 0 0.1"><joint name="filterexact"/><geom/></body>

    <body pos=" 0.5 0 0.1"><joint name="integrator_actearly"/><geom/></body>
    <body pos=" 0.7 0 0.1"><joint name="integrator"/><geom/></body>
  </worldbody>

  <actuator>
    <general class="filter" name="filter_actearly" joint="filter_actearly" actearly="true"/>
    <general class="filter" name="filter" joint="filter" actearly="false"/>

    <general class="filterexact" name="filterexact_actearly" joint="filterexact_actearly" actearly="true"/>
    <general class="filterexact" name="filterexact" joint="filterexact" actearly="false"/>

    <general class="integrator_actearly" name="integrator_actearly" joint="integrator_actearly"/>
    <general class="integrator" name="integrator" joint="integrator" actearly="false"/>
  </actuator>
</mujoco>
"""

xmlv6 = """
<mujoco>
  <compiler autolimits="true"/>
  <option density="1000"/>

  <worldbody>
    <geom type="plane" size="10 10 .1" rgba="1 1 1 1"/>
    <light pos="0 0 20"/>
    <site name="anchor_tendon" pos="0 0 9" size="0.03" rgba="0 0 1 1"/>
    <body name="body_hinge" pos="5 0 3">
      <geom type="box" size="1.5 .1 .1" pos="0 0 2.0"/>
      <joint name="hinge" type="hinge" pos="0 0 2.0" axis="0 1 0" range="-25 5"/>
    </body>
    <body name="body_tendon" pos="0 0 5">
      <freejoint/>
      <geom type="box" size=".5 .5 .5"/>
      <site name="box_tendon" pos=".5 .5 .5" rgba="0 0 1 1" size="0.03"/>
    </body>
    <body name="body_slider" pos="5 -2 2">
      <geom type="box" size=".5 .5 .5"/>
      <joint name="slider" type="slide" pos=".5 0 0" axis="1 0 0" range="-25 5"/>
    </body>
    <body name="body_site" pos="2 2 .5">
      <freejoint/>
      <geom type="box" size=".5 .5 .5"/>
      <site name="box_site" pos="0 0 .5" rgba="0 0 1 1" size="0.1"/>
    </body>
    <body name="body_stateless" pos="5 2 2">
      <geom type="box" size=".5 .5 .5"/>
      <joint name="stateless" type="slide" pos=".5 0 0" axis="1 0 0" range="-25 5"/>
    </body>
  </worldbody>

  <tendon>
    <spatial name="tendon" limited="true" range="0 4" damping="100"  width="0.03">
      <site site="anchor_tendon"/>
      <site site="box_tendon"/>
    </spatial>
  </tendon>

  <actuator>
    <general name="stateless" joint="stateless" ctrllimited="true" ctrlrange="-20 20"/>
    <general name="hinge" joint="hinge" dyntype="integrator" ctrllimited="true" ctrlrange="-20 20"/>
    <muscle name="tendon" tendon="tendon" ctrllimited="true" ctrlrange="0 2"/>
    <general name="slider" joint="slider" dyntype="integrator" ctrllimited="true" ctrlrange="-20 20"/>
    <general name="site" site="box_site" dyntype="integrator" gear="0 0 1 0 0 0" ctrllimited="true" ctrlrange="0 10000"/>
  </actuator>
</mujoco>"""


xmlv7 = """
<mujoco>
  <visual>
    <global elevation="-10"/>
  </visual>

  <compiler angle="radian"/>

  <!-- Note: actuator groups 1, 2, are disabled by default -->
  <option actuatorgroupdisable="1 2" cone="elliptic"/>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1=".3 .5 .7" rgb2="0 0 0" width="32" height="512"/>
    <texture name="body" type="cube" builtin="flat" mark="cross" width="128" height="128" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" markrgb="1 1 1" random="0.01"/>
    <material name="body" texture="body" texuniform="true" rgba="0.8 0.6 .4 1"/>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="6 2" texuniform="false" reflectance=".2"/>
  </asset>

  <default>
    <general ctrlrange="-1 1"/>
    <geom friction=".4"/>
    <default class="decor">
      <geom type="cylinder" size=".03" fromto="0 -.03 0 0 .03 0" rgba="1 1 1 1"/>
    </default>
    <default class="arm">
      <joint damping="5" axis="0 -1 0"/>
      <geom type="capsule" size=".02" material="body"/>
    </default>
    <default class="object">
      <geom type="cylinder" size="0.1" rgba="0 .9 0 1" fromto="0 -.03 0 0 .03 0"
            solref="-10000 -10" priority="1"/>
      <joint group="3"/>
    </default>

    <!-- Note: motors, position, intvelocity actuators are in groups 0, 1, 2, respectively -->
    <default class="motor">
      <motor group="0" gear="30" ctrlrange="-1 1"/>
    </default>
    <default class="position">
      <position group="1" kp="60"/>
    </default>
    <default class="intvelocity">
      <intvelocity group="2" kp="60" ctrlrange="-.5 .5"/>
    </default>
  </default>

  <worldbody>
    <geom name="floor" type="plane" size=".6 .2 0.01" material="grid"/>
    <geom name="wall0" type="plane" size=".4 .2 0.01" material="grid" pos="-.6 0 .4" zaxis="1 0 0"/>
    <geom name="wall1" type="plane" size=".4 .2 0.01" material="grid" pos=".6 0 .4" zaxis="-1 0 0"/>
    <light pos="-.6 0 3" mode="targetbody" target="hand"/>
    <light pos=".6 0 3" mode="targetbody" target="hand"/>
    <body name="arm" pos="-.6 0 .6" childclass="arm">
      <joint name="arm" range="-1.5 1.5"/>
      <geom class="decor"/>
      <geom name="arm" fromto="0 0 0 .7 0 0"/>
      <body pos=".7 0 0">
        <joint name="hand" range="-4.65 1.5"/>
        <geom class="decor"/>
        <geom fromto="0 0 0 0 0 0.4"/>
        <body name="hand" pos="0 0 0.4">
          <geom name="hand" type="sphere" size=".06" mass="0"/>
        </body>
      </body>
    </body>

    <body childclass="object" pos="-.3 0 .2">
      <joint axis="1 0 0" type="slide"/>
      <joint axis="0 0 1" type="slide"/>
      <joint axis="0 1 0"/>
      <geom/>
    </body>
    <body childclass="object" pos="-.2 0 .4">
      <joint axis="1 0 0" type="slide"/>
      <joint axis="0 0 1" type="slide"/>
      <joint axis="0 1 0"/>
      <geom/>
    </body>
  </worldbody>

  <contact>
    <exclude body1="world" body2="arm"/>
    <pair geom1="arm" geom2="hand"/>
  </contact>

  <actuator>
    <motor class="motor" name="arm torque" joint="arm"/>
    <motor class="motor" name="hand torque" joint="hand"/>
    <position class="position" name="arm position" joint="arm" ctrlrange="-1.5 1.5"/>
    <position class="position" name="hand position" joint="hand" ctrlrange="-1.5 4.65"/>
    <intvelocity class="intvelocity" name="arm velocity" joint="arm" actrange="-1.5 1.5"/>
    <intvelocity class="intvelocity" name="hand velocity" joint="hand" actrange="-1.5 4.65"/>
  </actuator>
</mujoco>

"""

#weld
xmlv8 = """
<mujoco>
  <option>
    <flag contact="disable"/>
  </option>

  <default>
    <default class="static">
      <geom size=".5 .1 .5" rgba=".5 .7 .5 .3"/>
    </default>
    <default class="free">
      <geom type="box" size=".2" fromto="0 0 0 0 -2 0" rgba=".4 .7 .6 .3"/>
    </default>
  </default>

  <worldbody>
    <geom pos="0 0 -2" type="plane" size="10 10 .01"/>
    <light pos="0 0 20"/>

    <body name="box1" pos="-3 0 0">
      <geom type="box" class="static"/>
    </body>
    <body name="beam1" pos="-3 0 0">
      <freejoint/>
      <geom class="free"/>
    </body>

    <body name="box2" pos="-1 0 0">
      <geom type="box" class="static"/>
    </body>
    <body name="beam2" pos="-1 0 0">
      <freejoint/>
      <geom class="free"/>
    </body>

    <body name="box3" pos="1 0 0">
      <geom type="box" class="static"/>
    </body>
    <body name="beam3" pos="1 0 0">
      <freejoint/>
      <geom class="free"/>
    </body>

    <body name="box4" pos="3 0 0">
      <geom type="box" class="static"/>
    </body>
    <body name="beam4" pos="3 0 0">
      <freejoint/>
      <geom class="free"/>
    </body>
  </worldbody>

  <equality>
    <weld name="weak torques"      body1="box1" body2="beam1" torquescale="0.002"/>
    <weld name="anchor no torques" body1="box2" body2="beam2" torquescale="0" anchor="0 -2 0"/>
    <weld name="relpose"           body1="box3" body2="beam3" relpose="0 0 0 1 -.3 0 0"/>
    <weld name="relpose+anchor"    body1="box4" body2="beam4" relpose="0 0 0 1 -.3 0 0" anchor="0 0 -1"/>
  </equality>
</mujoco>"""

#https://github.com/google-deepmind/mujoco/blob/main/test/engine/testdata/island/tendon_wrap.xml
xmlv9 = """
<mujoco>
  <option>
    <flag island="enable"/>
  </option>

  <default>
    <joint type="slide" axis="1 0 0" damping="3" frictionloss=".1"/>
    <geom type="cylinder" size=".07 .1" zaxis="0 1 0"/>
    <site size=".015" rgba="1 0 0 1"/>
  </default>

  <worldbody>
    <site name="anchor" pos="0 0 1"/>

    <body pos="0 0 .75">
      <joint stiffness="10"/>
      <geom name="0"/>
    </body>
    <site name="side0" pos=".15 0 .75" rgba="0 0 1 1"/>

    <site name="01" pos="0 0 .625"/>

    <body pos="0 0 .5">
      <joint stiffness="30"/>
      <geom name="1"/>
    </body>
    <site name="side1" pos="-.15 0 .5" rgba="0 0 1 1"/>

    <site name="12" pos="0 0 .375"/>

    <body pos="0 0 .25">
      <joint stiffness="100"/>
      <geom name="2"/>
    </body>
    <site name="side2" pos=".15 0 .25" rgba="0 0 1 1"/>

    <body>
      <freejoint/>
      <site name="box" pos="0 0 .1"/>
      <geom type="box" size=".1 .1 .1"/>
    </body>
  </worldbody>

  <tendon>
    <spatial limited="true" range="0 1.05" rgba="0 1 0 1" width=".005">
      <site site="anchor"/>
      <geom geom="0" sidesite="side0"/>
      <site site="01"/>
      <geom geom="1" sidesite="side1"/>
      <site site="12"/>
      <geom geom="2" sidesite="side2"/>
      <site site="box"/>
    </spatial>
  </tendon>
</mujoco>"""

xmlv10 = """
<mujoco model="tendon">
<default>
<joint axis="0 1 0" range="0 60" limited="true"/>
<geom type="capsule" size=".01" rgba=".9 .7 .1 1"/>
<site rgba=".9 .9 .9 1"/>
<default class="visual_cylinder">
<geom type="cylinder" fromto="0 .015 0 0 -.015 0" size=".02" rgba=".3 .9 .3 .4"/>
</default>
</default>
<worldbody>
<light pos="0 0 1"/>
<body name="weight" pos="-.03 0 .175">
<joint axis="0 0 1" pos="0 0 .025" type="slide"/>
<geom type="cylinder" size=".03 .025" density="5000" rgba=".2 .2 .5 1"/>
<site name="s1" pos="0 0 .025"/>
</body>
<site name="s2" pos="-.03 0 .33"/>
<body pos="0 0 .3">
<joint/>
<geom name="g1" class="visual_cylinder"/>
<geom fromto="0 0 0 .1 0 0"/>
<site name="s3" pos=".02 0 .03"/>
<body pos=".1 0 0">
<joint/>
<geom name="g2" class="visual_cylinder"/>
<geom fromto="0 0 0 .1 0 0"/>
<site name="s4" pos=".03 0 .01"/>
<site name="s5" pos=".05 0 .02"/>
<site name="side2" pos="0 0 .03"/>
<body pos=".1 0 0">
<joint/>
<geom name="g3" class="visual_cylinder"/>
<geom fromto="0 0 0 .1 0 0"/>
<site name="s6" pos=".03 0 .01"/>
<site name="side3" pos="0 0 .03"/>
</body>
</body>
</body>
</worldbody>
<tendon>
<spatial range="0 .33" limited="true" width=".002" rgba=".95 .3 .3 1">
<site site="s1"/>
<site site="s2"/>
<geom geom="g1"/>
<site site="s3"/>
<pulley divisor="2"/>
<site site="s3"/>
<geom geom="g2" sidesite="side2"/>
<site site="s4"/>
<pulley divisor="2"/>
<site site="s3"/>
<geom geom="g2" sidesite="side2"/>
<site site="s5"/>
<geom geom="g3" sidesite="side3"/>
<site site="s6"/>
</spatial>
</tendon>
</mujoco>"""

#https://github.com/google-deepmind/mujoco/issues/803
xmlv11= """
<mujoco>
  <compiler autolimits="true"/>

  <default>
    <site type="sphere" rgba="1 0 0 1" size="0.005"/>
    <tendon rgba="0 1 0 1"/>
  </default>
  
  <visual>
    <headlight diffuse=".7 .7 .7"/>
  </visual>

  <worldbody>
    <body name="arm">
      <joint name="arm" axis="0 1 0" damping=".04"/>
      <geom name="shoulder" type="cylinder" size=".05 .01" zaxis="0 1 0"/>
      <geom name="arm" type="box" size=".01" fromto="0 0 0 -.3 0 0"/>
      <geom name="crossbar" type="box" size=".01" fromto="-.1 0 -.05 -.1 0 .05"/>
      <site name="arm" pos="-.1 0 .05"/>
    </body>
    <site name="right" pos=".1 0 0"/>
    <body name="slider" pos=".05 0 -.2">
      <joint name="slider" type="slide" damping="1"/>
      <geom name="slider" type="box" size=".01 .01 .01"/>
      <site name="slider" pos="0 0 .01"/>
    </body>
  </worldbody>

  <tendon>
    <spatial name="rope" range="0 .35">
      <site site="slider"/>
      <geom geom="shoulder" sidesite="right"/>
      <site site="arm"/>
    </spatial>
  </tendon>

  <actuator>
    <position name="slider" joint="slider" ctrlrange="-.05 .05" kp="100"/>
  </actuator>
</mujoco>
"""

#https://github.com/google-deepmind/mujoco/blob/main/test/engine/testdata/tendon_springlength.xml
xmlv12 = """
<mujoco>
  <!-- use Runge-Kutta for better energy conservation -->
  <option integrator="RK4">
    <flag energy="enable"/>
  </option>

  <statistic center="0 0 0"/>

  <worldbody>
    <site name="origin" size=".01" rgba="1 0 0 1"/>
    <light pos="0 0 3"/>

    <body pos="1 0 .06">
      <joint type="slide" axis="1 0 0"/>
      <geom size=".05"/>
      <site name="attachment" size=".01" pos="0 0 -.06" rgba="1 0 0 1"/>
    </body>
  </worldbody>

  <tendon>
    <spatial name="tendon" stiffness="10" springlength="0 .5">
      <site site="origin"/>
      <site site="attachment"/>
    </spatial>
  </tendon>

  <sensor>
    <tendonpos tendon="tendon"/>
  </sensor>
</mujoco>"""

In [9]:
xml5 = """
<mujoco>
    <compiler autolimits="true"/>
    <asset>
        <!-- Define ground material -->
        <material name="ground" rgba="0.8 0.9 0.8 1"/>
    </asset>

    <option gravity ="0 0 0"/>
    
  <default>
    <default class="translation">
      <position kp="100" kv="10" ctrlrange="-.5 .5"/>
    </default>
    <default class="rotation">
      <position kp=".5" kv=".05" ctrlrange="-1.571 1.571"/>
    </default>
    <joint stiffness="1e-1" actuatorfrcrange="-1 1"/>
  </default>

  <option viscosity="10" integrator="implicit">
    <flag contact="disable"/>
  </option>

    <visual>
      <headlight ambient="0.5 0.5 0.5"/>
    </visual>

    <worldbody>
        <!-- Define the ground -->
        <geom name="ground" type="plane" size="1 1 0.1" material="ground"/>

        <!-- Define the box -->
        <body name="box" pos="0 0 0.5">
            <!--freejoint/-->
            <joint name="pin" type="ball" axis = "0 -1 0"/>
            <geom name="box_geom" type="box" size="0.1 0.1 0.1"/>
            <site name="box"/>
            <body name="rod1" pos="0 -0.1 0.5" euler="0 0 0">
              <joint name="pin2" type="hinge" axis = "0 -1 0" pos="0 0 -0.5"/>
				      <geom type="cylinder" size="0.05 0.5" rgba="0 0 .9 1" mass="1"/>
              <site name="rod1"/>
              <joint name="pin3" type="slide" axis = "0 0 1"/>
            </body>
        </body>
    </worldbody>
    
    <equality>
        <connect name="connect" body1="box" body2="rod1" anchor="0 0 0.5"/>
        <!--weld name="weld1" body1="rod12" anchor="0 0 1.5"/-->
        <!--weld name="weld2" body1="rod1" body2="rod12"/-->
    </equality> 

  <actuator>
    <position name="move_rod" site="rod1" gear="0 0 1 0 0 0" class="translation"/>
    <motor name="translate_rod" joint="pin3"/>
  </actuator>
    
</mujoco>
"""

model = mu.MjModel.from_xml_string(xmlv12)

data = mu.MjData(model)
mu.mj_kinematics(model,data)

duration = 10 # (seconds)
framerate = 60  # (Hz)

# enable joint visualization option:
scene_option = mu.MjvOption()
scene_option.flags[mu.mjtVisFlag.mjVIS_JOINT] = True
rend = mu.Renderer(model)

vi.launch(model)

In [10]:
# Simulate and display video.
frames = []
mu.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mu.mj_step(model, data)
  if len(frames) < data.time * framerate:
    rend.update_scene(data)
    pixels = rend.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

RuntimeError: Program 'ffmpeg' is not found; perhaps install ffmpeg using 'apt install ffmpeg'.